In [1]:
import pandas as pd
import numpy as np
import librosa

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv1D,
    MaxPooling1D,
    Flatten,
    Dropout,
    BatchNormalization,
)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

### Read dataset

In [2]:
train_dataset = pd.read_excel("./datasets/train_dataset.xlsx")
test_dataset = pd.read_excel("./datasets/test_dataset.xlsx")

### Data Augmentation

In [ ]:
def noise(data):
    """
    Add random Gaussian noise to the audio signal for data augmentation.

    This function adds controlled random noise to the input audio signal. The amplitude
    of the noise is proportional to the maximum amplitude of the input signal,
    making it adaptive to different audio volumes.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal time series

    Returns
    -------
    numpy.ndarray
        Audio signal with added noise, same shape as input

    Notes
    -----
    The noise generation process:
    1. Calculate noise amplitude as 3.5% of input signal's max amplitude
    2. Generate Gaussian noise with the same length as input
    3. Scale noise by calculated amplitude
    4. Add scaled noise to original signal

    The noise amplitude is randomized using uniform distribution to create
    variety in the augmented data.
    """
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)
    data = data + noise_amp * np.random.normal(size=data.shape[0])
    return data


def stretch(data, rate=0.8):
    """
    Time-stretch the audio signal without changing its pitch.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal
    rate : float, optional
        Stretching rate. Values > 1 speed up the audio, values < 1 slow it down.
        Default is 0.8 (20% slower).

    Returns
    -------
    numpy.ndarray
        Time-stretched audio signal
    """
    return librosa.effects.time_stretch(data, rate=rate)


def shift(data):
    """
    Randomly shift the audio signal in time.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal

    Returns
    -------
    numpy.ndarray
        Time-shifted audio signal, shifted by -5000 to 5000 samples
    """
    shift_range = int(np.random.uniform(low=-5, high=5) * 1000)
    return np.roll(data, shift_range)


def pitch(data, sampling_rate, pitch_factor=0.7):
    """
    Shift the pitch of the audio signal.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal
    sampling_rate : int
        Sampling rate of the input audio
    pitch_factor : float, optional
        Number of semitones to shift. Default is 0.7 (lower pitch)

    Returns
    -------
    numpy.ndarray
        Pitch-shifted audio signal
    """
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)

We use only noise and stretch, copying the steps from kaggle notebook

In [ ]:
def extract_audio_features(data):
    """
    Extract audio features from the input audio data for emotion recognition.

    This function extracts multiple audio features that are useful for speech emotion recognition:
    - Zero Crossing Rate (ZCR): Rate at which the signal changes from positive to negative
    - Chroma STFT: Represents the spectral energy across the 12 pitch classes
    - MFCC (Mel-frequency cepstral coefficients): Represents the short-term power spectrum
    - RMS (Root Mean Square): Represents the loudness of the signal
    - Mel Spectrogram: Represents the power spectral density on mel-scale

    Parameters
    ----------
    data : numpy.ndarray
        Audio time series data loaded using librosa

    Returns
    -------
    numpy.ndarray
        1D array containing concatenated features in the following order:
        [ZCR, Chroma STFT, MFCC, RMS, Mel Spectrogram]

    Notes
    -----
    All features are averaged across time using mean to get a fixed-length
    representation regardless of the input audio length.
    """
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))  # stacking horizontally

    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))

    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc))

    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))

    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel))

    return result


def augment_and_get_features(path):
    """
    Load an audio file, apply data augmentation, and extract features.

    This function performs the following steps:
    1. Loads the audio file
    2. Extracts features from the original audio
    3. Applies noise augmentation and extracts features
    4. Applies time stretching followed by pitch shifting and extracts features

    Parameters
    ----------
    path : str
        Path to the audio file

    Returns
    -------
    numpy.ndarray
        2D array of shape (3, n_features) containing features from:
        - Row 0: Original audio
        - Row 1: Noise augmented audio
        - Row 2: Stretch and pitch augmented audio
    """
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path)

    # without augmentation
    res1 = extract_audio_features(data)
    result = np.array(res1)

    # data with noise
    noise_data = noise(data)
    res2 = extract_audio_features(noise_data)
    result = np.vstack((result, res2))  # stacking vertically

    # data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_audio_features(data_stretch_pitch)
    result = np.vstack((result, res3))  # stacking vertically

    return result


def get_features(path):
    """
    Load an audio file, apply data augmentation, and extract features.

    This function performs the following steps:
    1. Loads the audio file
    2. Extracts features from the original audio

    Parameters
    ----------
    path : str
        Path to the audio file

    Returns
    -------
    numpy.ndarray
        2D array of shape (1, n_features) containing features from:
        - Row 0: Original audio
    """
    data, sample_rate = librosa.load(path)

    result = extract_audio_features(data)
    result = np.array(result)

    return result

In [ ]:
x_train = train_dataset["Path"]
y_train = train_dataset["Emotion"]
x_test = test_dataset["Path"]
y_test = test_dataset["Emotion"]

In [ ]:
X, Y = [], []
for path, emotion in zip(x_train, y_train):
    feature = augment_and_get_features(path)
    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [ ]:
len(X), len(Y)

In [ ]:
Features = pd.DataFrame(X)
Features["labels"] = Y
Features.head(2)

### Data Preparation

In [ ]:
# test_dataset = pd.read_csv("./datasets/test_dataset.csv")
# y = test_dataset[["Emotion"]]
# x = test_dataset.drop("Emotion", axis=1)
# x.shape, y.shape

In [ ]:
def get_features_from_test(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)

    # without augmentation
    res = extract_audio_features(data)
    result = np.array(res)

    return result

In [ ]:
x_test, y_test = [], []
for path, emotion in zip(x.Path, y.Emotion):
    features = get_features_from_test(path)
    x_test.append(features)
    y_test.append(emotion)

In [ ]:
x_test, y_test

In [ ]:
train_dataset = pd.read_csv("./datasets/train_dataset.csv")

# Separate features (X) and labels (Y)
Y = train_dataset[["labels"]].copy()
# Y.columns = ['Emotion']  # Rename the column to match our convention
X = train_dataset.drop("labels", axis=1)
X.shape, Y.shape

In [ ]:
test_dataset = pd.read_csv("./datasets/test_dataset.csv")

y_test = test_dataset[["Emotion"]].copy()
x_test = test_dataset.drop("Emotion", axis=1)
x_test.shape, y_test.shape

One Hot Encoding

In [ ]:
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1)).toarray()
# y_test = encoder.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [ ]:
y_test = encoder.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [ ]:
Y, encoder.categories_

In [ ]:
y_test, encoder.categories_

Scaler

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

In [ ]:
x_test = scaler.fit_transform(x_test)
x_test

In [ ]:
X.shape

In [ ]:
X = np.expand_dims(X, axis=2)
X.shape

### Modelling

In [ ]:
model = Sequential()
model.add(
    Conv1D(
        256,
        kernel_size=5,
        strides=1,
        padding="same",
        activation="relu",
        input_shape=(X.shape[1], 1),
    )
)
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))

model.add(Conv1D(256, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))

model.add(Conv1D(128, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))

model.add(Flatten())
model.add(Dense(units=32, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(units=8, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
rlrp = ReduceLROnPlateau(
    monitor="loss", factor=0.4, verbose=0, patience=2, min_lr=10e-8
)
history = model.fit(
    X, Y, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[rlrp]
)